# Stable Diffusion 3.5 Medium - 3-Level Grass Species Generation

Generating scientific documentation images for grass species with **3 optimized levels** for more consistent results.

## Species and Conditions (3-Level System):
- **Brachiaria brizantha**: Saúde Ótima → Boa Condição → Estresse Moderado
- **Panicum maximum**: Vigor Excelente → Crescimento Adequado → Vigor em Declínio
- **Cynodon dactylon**: Condição Premium → Qualidade Padrão → Cobertura Rarefeita

**Total**: 3 species × 3 conditions × 2 images = 18 images

**Key Improvements**:
- Reduced from 5 to 3 levels for more consistent generation
- Refined prompts for better species recognition
- Enhanced unique file naming to prevent overwrites

In [ ]:
# Install required packages
!pip install diffusers transformers accelerate torch torchvision huggingface_hub --quiet

In [ ]:
# Import libraries
import torch
from diffusers import StableDiffusion3Pipeline
from huggingface_hub import login
import matplotlib.pyplot as plt
from PIL import Image
import os
from datetime import datetime

In [ ]:
# Check GPU availability
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

In [ ]:
# Hugging Face login (required for SD 3.5 Medium)
login()

In [ ]:
# Load the Stable Diffusion 3.5 Medium model
print("Loading Stable Diffusion 3.5 Medium...")
pipe = StableDiffusion3Pipeline.from_pretrained(
    "stabilityai/stable-diffusion-3.5-medium",
    torch_dtype=torch.float16,
    device_map="balanced"
)

print("Model loaded successfully!")

In [ ]:
# Sistema de 3 níveis otimizado para melhores resultados
brachiaria_prompts_refined = {
    'brachiaria_optimal_health': {
        'positive': (
            "perfect overhead view of prime brachiaria brizantha pasture, "
            "thick vibrant green tropical grass, dense uniform coverage, "
            "wide leaf blades in natural tufts, robust plant structure, "
            "excellent ground coverage, no bare soil visible, "
            "agricultural research photography, natural daylight, "
            "top-down perspective, healthy tropical grassland"
        ),
        'negative': (
            "sparse coverage, thin grass, bare patches, yellow grass, "
            "ground level view, side angle, artificial, synthetic, "
            "ornamental lawn, temperate species, flowers"
        ),
        'description': "Brachiaria - Saúde Ótima"
    },
    
    'brachiaria_good_condition': {
        'positive': (
            "aerial view of good condition brachiaria pasture, "
            "mostly green grass with natural color variation, "
            "good coverage with minor irregular patches, "
            "wide leaf tropical grass showing normal field variation, "
            "scientific documentation, natural lighting, "
            "overhead perspective, realistic pasture condition"
        ),
        'negative': (
            "severely damaged, mostly bare soil, dying grass, "
            "ground perspective, ornamental setting, artificial, "
            "temperate grass, decorative lawn"
        ),
        'description': "Brachiaria - Boa Condição"
    },
    
    'brachiaria_moderate_stress': {
        'positive': (
            "top-down view of brachiaria pasture under moderate stress, "
            "mixed green and lighter colored grass areas, "
            "uneven plant density, some small bare soil spots visible, "
            "tropical grass showing stress but still recognizable, "
            "natural variation in coverage, field research documentation, "
            "realistic pasture with environmental stress"
        ),
        'negative': (
            "completely healthy, uniform green, perfect coverage, "
            "ground level view, decorative grass, artificial, "
            "abstract patterns, unrecognizable vegetation"
        ),
        'description': "Brachiaria - Estresse Moderado"
    }
}

# Panicum com 3 níveis
panicum_prompts_refined = {
    'panicum_excellent_vigor': {
        'positive': (
            "overhead view of vigorous panicum maximum pasture, "
            "tall robust green tropical grass, wide leaf blades, "
            "dense natural clumping pattern, excellent coverage, "
            "healthy upright growth, thick stems visible, "
            "research quality photography, natural lighting"
        ),
        'negative': (
            "short grass, thin blades, sparse coverage, ground view, "
            "artificial, ornamental, temperate species"
        ),
        'description': "Panicum - Vigor Excelente"
    },
    
    'panicum_adequate_growth': {
        'positive': (
            "aerial view of adequate panicum grass field, "
            "mostly green tall grass with natural variation, "
            "good but not perfect coverage, mixed growth vigor, "
            "large tropical grass with normal field irregularities, "
            "scientific documentation, top-down perspective"
        ),
        'negative': (
            "severely stressed, mostly bare, dying vegetation, "
            "ground perspective, decorative setting, artificial"
        ),
        'description': "Panicum - Crescimento Adequado"
    },
    
    'panicum_declining_vigor': {
        'positive': (
            "overhead documentation of declining panicum pasture, "
            "tall grass with mixed green and pale areas, "
            "reduced plant density, some bare patches, "
            "tropical grass showing stress but maintaining structure, "
            "natural field conditions, research photography"
        ),
        'negative': (
            "perfect health, uniform coverage, decorative lawn, "
            "ground view, abstract patterns, unrecognizable"
        ),
        'description': "Panicum - Vigor em Declínio"
    }
}

# Cynodon com 3 níveis
cynodon_prompts_refined = {
    'cynodon_premium_condition': {
        'positive': (
            "perfect overhead view of premium cynodon dactylon, "
            "dense carpet-like grass coverage, uniform bright green, "
            "fine textured stoloniferous mat, excellent ground coverage, "
            "tight formation, no visible soil, research quality, "
            "natural lighting, top-down agricultural documentation"
        ),
        'negative': (
            "sparse coverage, bare patches, coarse texture, "
            "ground level view, artificial turf, ornamental"
        ),
        'description': "Cynodon - Condição Premium"
    },
    
    'cynodon_standard_quality': {
        'positive': (
            "aerial view of standard quality cynodon pasture, "
            "good carpet coverage with minor thin areas, "
            "mostly uniform green with natural color variation, "
            "stoloniferous grass with adequate density, "
            "scientific field documentation, overhead perspective"
        ),
        'negative': (
            "severely thin, mostly bare soil, dying grass, "
            "ground view, decorative setting, artificial"
        ),
        'description': "Cynodon - Qualidade Padrão"
    },
    
    'cynodon_thinning_coverage': {
        'positive': (
            "top-down view of cynodon with thinning coverage, "
            "patchy grass areas with visible thin spots, "
            "mixed density stoloniferous growth, some soil showing, "
            "declining but still recognizable cynodon pasture, "
            "natural field conditions, research documentation"
        ),
        'negative': (
            "thick perfect carpet, uniform coverage, decorative turf, "
            "ground perspective, abstract patterns, unrecognizable"
        ),
        'description': "Cynodon - Cobertura Rarefeita"
    }
}

# Combine all prompts for the 3-level system
all_prompts_3level = {**brachiaria_prompts_refined, **panicum_prompts_refined, **cynodon_prompts_refined}

print(f"Loaded {len(brachiaria_prompts_refined)} Brachiaria prompts (3-level system)")
print(f"Loaded {len(panicum_prompts_refined)} Panicum prompts (3-level system)")
print(f"Loaded {len(cynodon_prompts_refined)} Cynodon prompts (3-level system)")
print(f"Total prompts: {len(all_prompts_3level)}")
print("✓ 3-Level optimized prompts loaded for more consistent results")

In [ ]:
# Create output directory with timestamp and date organization (unique naming system)
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
date_folder = datetime.now().strftime("%Y-%m-%d")
output_dir = f"grass_3level_generation_{timestamp}"
os.makedirs(output_dir, exist_ok=True)
print(f"Output directory: {output_dir}")
print(f"Date folder for organization: {date_folder}")
print(f"Unique timestamp: {timestamp}")
print("✓ Each execution will have unique file names to prevent overwrites")

In [ ]:
# Function to generate images with optimized parameters for 3-level system
import random

def generate_species_images_3level(prompt_data, num_images=2, execution_id=None):
    """
    Generate images using 3-level optimized parameters with unique naming
    """
    images = []
    
    positive_prompt = prompt_data['positive']
    negative_prompt = prompt_data['negative']
    
    print(f"Generating {num_images} images for: {prompt_data['description']}")
    print(f"Positive: {positive_prompt[:100]}...")
    print(f"Negative: {negative_prompt[:100]}...")
    
    for i in range(num_images):
        print(f"  Generating image {i+1}/{num_images}...")
        
        # Optimized parameters for 3-level system (more controlled variation)
        guidance_scale = random.uniform(5.0, 6.0)  # Tighter range for consistency
        num_steps = random.randint(30, 32)         # Consistent quality range
        seed = random.randint(0, 2**32 - 1)       # Random seed for variety
        
        # Less prompt variation for more consistent results
        prompt_variations = [
            positive_prompt,
            positive_prompt + ", natural field variation"
        ]
        
        selected_prompt = random.choice(prompt_variations)
        
        print(f"    Parameters: guidance={guidance_scale:.1f}, steps={num_steps}, seed={seed}")
        
        with torch.inference_mode():
            # Set the generator with our random seed
            generator = torch.Generator(device="cuda" if torch.cuda.is_available() else "cpu")
            generator.manual_seed(seed)
            
            image = pipe(
                prompt=selected_prompt,
                negative_prompt=negative_prompt,
                num_inference_steps=num_steps,
                guidance_scale=guidance_scale,
                height=1024,
                width=1024,
                generator=generator
            ).images[0]
        
        images.append(image)
    
    return images

# Function to determine species from prompt key
def get_species_from_key(prompt_key):
    if prompt_key.startswith('brachiaria'):
        return 'brachiaria'
    elif prompt_key.startswith('panicum'):
        return 'panicum'
    elif prompt_key.startswith('cynodon'):
        return 'cynodon'
    else:
        return 'unknown'

print("✓ 3-Level generation functions loaded with optimized parameters")

In [ ]:
# Test 3-level generation with a sample prompt
print("Testing 3-level generation system...")

# Select a test prompt from the 3-level system
test_prompt_key = 'brachiaria_optimal_health'
test_prompt_data = all_prompts_3level[test_prompt_key]

# Generate a test image with 3-level parameters
print("Generating test images with 3-level optimized parameters...")
test_images = generate_species_images_3level(test_prompt_data, num_images=1, execution_id=timestamp)

# Display the test result
if test_images:
    import matplotlib.pyplot as plt
    plt.figure(figsize=(8, 8))
    plt.imshow(test_images[0])
    plt.title(f"Test Result (3-Level): {test_prompt_data['description']}")
    plt.axis('off')
    plt.show()
    print("✓ 3-Level test generation completed successfully!")
else:
    print("✗ Test generation failed")

In [ ]:
# Generate all images for all species using 3-level system
all_generated_images_3level = {}
total_prompts = len(all_prompts_3level)
current_prompt = 0

for prompt_key, prompt_data in all_prompts_3level.items():
    current_prompt += 1
    species = get_species_from_key(prompt_key)
    print(f"\n{'='*50}")
    print(f"Processing {current_prompt}/{total_prompts}: {prompt_key} ({species})")
    print(f"3-Level System - {prompt_data['description']}")
    print(f"{'='*50}")
    
    try:
        images = generate_species_images_3level(prompt_data, num_images=2, execution_id=timestamp)
        all_generated_images_3level[prompt_key] = {
            'images': images,
            'description': prompt_data['description'],
            'species': species,
            'execution_id': timestamp
        }
        print(f"✓ Successfully generated 2 images for {prompt_key}")
        
    except Exception as e:
        print(f"✗ Error generating {prompt_key}: {e}")
        all_generated_images_3level[prompt_key] = {
            'images': [],
            'description': prompt_data['description'],
            'species': species,
            'error': str(e),
            'execution_id': timestamp
        }

print(f"\n\n3-LEVEL GENERATION COMPLETE!")
successful_sets = len([k for k, v in all_generated_images_3level.items() if v['images']])
print(f"Generated images for {successful_sets} prompt sets out of {total_prompts}")

# Count by species
species_count = {}
for data in all_generated_images_3level.values():
    species = data['species']
    if species not in species_count:
        species_count[species] = {'total': 0, 'successful': 0}
    species_count[species]['total'] += 1
    if data['images']:
        species_count[species]['successful'] += 1

for species, counts in species_count.items():
    print(f"{species.capitalize()}: {counts['successful']}/{counts['total']} sets successful (3 levels each)")

In [ ]:
# Display 3-level images in organized grid layout
def display_3level_image_grid(all_images, cols=2):
    """
    Display all generated 3-level images in a organized grid layout by species
    """
    # Organize by species
    species_data = {}
    for prompt_key, data in all_images.items():
        if data['images']:
            species = data['species']
            if species not in species_data:
                species_data[species] = []
            species_data[species].append((prompt_key, data))
    
    if not species_data:
        print("No images to display")
        return
    
    # Create separate grids for each species
    for species, prompt_list in species_data.items():
        rows = len(prompt_list)
        
        fig, axes = plt.subplots(rows, cols, figsize=(15, 5*rows))
        if rows == 1:
            axes = axes.reshape(1, -1)
        
        fig.suptitle(f"3-Level {species.capitalize()} Generation Results", fontsize=16, y=0.98)
        
        for i, (prompt_key, data) in enumerate(prompt_list):
            images = data['images']
            description = data['description']
            
            for j in range(min(cols, len(images))):
                axes[i, j].imshow(images[j])
                axes[i, j].set_title(f"{description} - Img {j+1}", fontsize=10)
                axes[i, j].axis('off')
            
            # Hide unused subplots
            for j in range(len(images), cols):
                axes[i, j].axis('off')
        
        plt.tight_layout()
        grid_filename = f"{output_dir}/{species}_3level_grid_{timestamp}.png"
        plt.savefig(grid_filename, dpi=300, bbox_inches='tight')
        plt.show()
        
        print(f"Grid saved: {grid_filename}")

# Display the 3-level grids
display_3level_image_grid(all_generated_images_3level)

In [ ]:
# Save individual images with unique naming system
def save_3level_images_with_unique_names(all_images, output_directory, execution_timestamp):
    """
    Save each generated image individually with unique names to prevent overwrites
    Format: {species}/{species}_{condition}_{timestamp}_{image_number}.png
    """
    saved_count = 0
    species_folders = {}
    
    # Create species directories
    for prompt_key, data in all_images.items():
        species = data['species']
        if species not in species_folders:
            species_dir = os.path.join(output_directory, species)
            os.makedirs(species_dir, exist_ok=True)
            species_folders[species] = species_dir
            print(f"Created directory: {species_dir}")
    
    # Save images with unique naming
    for prompt_key, data in all_images.items():
        if not data['images']:
            continue
        
        species = data['species']
        species_dir = species_folders[species]
        
        # Extract condition from prompt_key for better naming
        condition_map = {
            'optimal_health': 'saude_otima',
            'good_condition': 'boa_condicao', 
            'moderate_stress': 'estresse_moderado',
            'excellent_vigor': 'vigor_excelente',
            'adequate_growth': 'crescimento_adequado',
            'declining_vigor': 'vigor_declinio',
            'premium_condition': 'condicao_premium',
            'standard_quality': 'qualidade_padrao',
            'thinning_coverage': 'cobertura_rarefeita'
        }
        
        condition = 'unknown'
        for key, value in condition_map.items():
            if key in prompt_key:
                condition = value
                break
        
        for i, image in enumerate(data['images']):
            # Unique filename format: species_condition_timestamp_imageN.png
            filename = f"{species}_{condition}_{execution_timestamp}_img{i+1}.png"
            filepath = os.path.join(species_dir, filename)
            
            image.save(filepath)
            saved_count += 1
            print(f"Saved: {species}/{filename}")
    
    print(f"\nTotal images saved with unique names: {saved_count}")
    print(f"Images organized in {len(species_folders)} species folders")
    print(f"Naming format: species_condition_{execution_timestamp}_imgN.png")
    return saved_count, species_folders

# Save all images with unique naming system
saved_images_3level, species_directories_3level = save_3level_images_with_unique_names(
    all_generated_images_3level, output_dir, timestamp
)

In [ ]:
# Generate comprehensive 3-level summary report
print("\n" + "="*60)
print("3-LEVEL GRASS SPECIES GENERATION SUMMARY")
print("="*60)
print(f"Execution ID: {timestamp}")
print(f"Date: {date_folder}")

successful_prompts = 0
total_images = 0

# Detailed summary by species and levels
level_names = {
    'brachiaria': ['Saúde Ótima', 'Boa Condição', 'Estresse Moderado'],
    'panicum': ['Vigor Excelente', 'Crescimento Adequado', 'Vigor em Declínio'],
    'cynodon': ['Condição Premium', 'Qualidade Padrão', 'Cobertura Rarefeita']
}

for species in ['brachiaria', 'panicum', 'cynodon']:
    species_prompts = [k for k, v in all_generated_images_3level.items() if v['species'] == species]
    species_successful = [k for k in species_prompts if all_generated_images_3level[k]['images']]
    species_total_images = sum(len(all_generated_images_3level[k]['images']) for k in species_prompts)
    
    print(f"\n{species.upper()} (3 LEVELS):")
    for i, prompt_key in enumerate(species_prompts):
        data = all_generated_images_3level[prompt_key]
        level_num = i + 1
        if data['images']:
            print(f"  ✓ Nível {level_num}: {data['description']} - {len(data['images'])} images")
            successful_prompts += 1
            total_images += len(data['images'])
        else:
            print(f"  ✗ Nível {level_num}: {data['description']} - Failed: {data.get('error', 'Unknown error')}")

print(f"\nOVERALL 3-LEVEL RESULTS:")
print(f"- Successful prompt sets: {successful_prompts}/{len(all_prompts_3level)}")
print(f"- Total images generated: {total_images}")
print(f"- Species processed: {len(species_directories_3level)}")
print(f"- Levels per species: 3 (optimized system)")
print(f"- Images per level: 2")
print(f"- Expected total: 18 images (3 species × 3 levels × 2 images)")
print(f"- Output directory: {output_dir}")
print(f"- Species folders: {list(species_directories_3level.keys())}")
print(f"- Execution timestamp: {timestamp}")
print(f"- File naming format: species_condition_{timestamp}_imgN.png")
print("="*60)

In [ ]:
# Mount Google Drive for organized storage
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import shutil
import os

# Function to create organized Google Drive structure for 3-level system
def create_3level_google_drive_structure(local_output_dir, species_directories, date_folder, execution_timestamp):
    """
    Create organized 3-level folder structure in Google Drive:
    img_gen_3level/
    ├── brachiaria/
    │   └── 2024-08-28/
    │       └── execution_20240828_143022/
    │           └── images (3 levels × 2 images = 6 images)
    ├── panicum/
    │   └── 2024-08-28/
    │       └── execution_20240828_143022/
    │           └── images (3 levels × 2 images = 6 images)
    └── cynodon/
        └── 2024-08-28/
            └── execution_20240828_143022/
                └── images (3 levels × 2 images = 6 images)
    """
    
    # Base Google Drive directory for 3-level system
    base_drive_dir = '/content/drive/MyDrive/img_gen_3level'
    os.makedirs(base_drive_dir, exist_ok=True)
    
    print(f"Creating 3-level organized structure in Google Drive: {base_drive_dir}")
    print(f"Date folder: {date_folder}")
    print(f"Execution timestamp: {execution_timestamp}")
    
    copied_files = 0
    execution_folder = f"execution_{execution_timestamp}"
    
    for species, local_species_dir in species_directories.items():
        # Create species folder in Google Drive
        drive_species_dir = os.path.join(base_drive_dir, species)
        os.makedirs(drive_species_dir, exist_ok=True)
        
        # Create date folder within species
        drive_date_dir = os.path.join(drive_species_dir, date_folder)
        os.makedirs(drive_date_dir, exist_ok=True)
        
        # Create execution folder within date (to prevent overwrites)
        drive_execution_dir = os.path.join(drive_date_dir, execution_folder)
        os.makedirs(drive_execution_dir, exist_ok=True)
        
        print(f"Created: {drive_execution_dir}")
        
        # Copy all images from local species directory to Google Drive
        if os.path.exists(local_species_dir):
            for filename in os.listdir(local_species_dir):
                if filename.endswith('.png'):
                    local_file = os.path.join(local_species_dir, filename)
                    drive_file = os.path.join(drive_execution_dir, filename)
                    shutil.copy2(local_file, drive_file)
                    copied_files += 1
                    print(f"  Copied: {species}/{date_folder}/{execution_folder}/{filename}")
    
    # Copy grid overview images to base directory
    for species in species_directories.keys():
        grid_overview_local = os.path.join(local_output_dir, f"{species}_3level_grid_{execution_timestamp}.png")
        if os.path.exists(grid_overview_local):
            grid_overview_drive = os.path.join(base_drive_dir, f"{species}_3level_overview_{date_folder}_{execution_timestamp}.png")
            shutil.copy2(grid_overview_local, grid_overview_drive)
            print(f"Copied grid overview: {species}_3level_overview_{date_folder}_{execution_timestamp}.png")
    
    return base_drive_dir, copied_files, execution_folder

# Create organized Google Drive structure for 3-level system
try:
    drive_base_dir, total_copied, exec_folder = create_3level_google_drive_structure(
        output_dir, species_directories_3level, date_folder, timestamp
    )
    
    print(f"\n{'='*60}")
    print(f"3-LEVEL GOOGLE DRIVE UPLOAD SUMMARY")
    print(f"{'='*60}")
    print(f"✓ Successfully copied {total_copied} images to Google Drive")
    print(f"✓ Base directory: {drive_base_dir}")
    print(f"✓ Organization: Species > Date > Execution > Images")
    print(f"✓ Date folder: {date_folder}")
    print(f"✓ Execution folder: {exec_folder}")
    print(f"✓ Unique naming prevents overwrites between runs")
    
    # Display folder structure
    print(f"\n3-Level folder structure created:")
    for species in species_directories_3level.keys():
        print(f"  img_gen_3level/{species}/{date_folder}/{exec_folder}/")
        species_images = len([f for f in os.listdir(species_directories_3level[species]) if f.endswith('.png')])
        print(f"    └── {species_images} images (3 levels × 2 images each)")
    
    print(f"{'='*60}")
    
except Exception as e:
    print(f"Error creating 3-level Google Drive structure: {e}")
    print(f"Attempting simple backup copy...")
    
    # Fallback: simple copy with timestamp
    try:
        simple_drive_dir = f'/content/drive/MyDrive/img_gen_3level_backup_{timestamp}'
        shutil.copytree(output_dir, simple_drive_dir, dirs_exist_ok=True)
        print(f"Backup copy successful: {simple_drive_dir}")
    except Exception as backup_error:
        print(f"Backup copy also failed: {backup_error}")